In [150]:
from py2neo import Graph
from neo4j import GraphDatabase
import pandas as pd
import numpy as np 
import os
from dotenv import load_dotenv
import logging

## Load Enviornment Variables

In [151]:
load_dotenv()
USERNAME = os.getenv('USERNAME_NEO4J')
PASS = os.getenv('PASSWORD_NEO4J')
URI = 'neo4j://172.19.30.125:7687'

## Configuring Logging

In [152]:
import logging 

# Configure logging
logging.basicConfig(filename='neo4j_import.log',
                     level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


## Defining Graph Properties for Neo4j

### Cypher Queries - Graph Schema

In [191]:
def load_data_from_csv(csv_file):

    cypher_query = """
    LOAD CSV WITH HEADERS FROM $csv_file AS row
    WITH row
    WHERE row.FLAG = '1'
    MERGE (sc:SmartContract {contractAddress: row.contractAddress})
    ON CREATE SET sc.reputable = toBoolean(row.FLAG)

    WITH row
    MERGE (a1:Account {address: row.from})
    SET a1.balance = toFloat(row.balance1)
    MERGE (a2:Account {address: row.to})
    SET a2.balance = toFloat(row.balance2)

    WITH row
    MERGE (b:Block {blockNumber: toInteger(row.blockNumber)})
    ON CREATE SET b.timeStamp = toInteger(row.timeStamp)

    WITH row
    MATCH (from:Account {address: row.from}), (to:Account {address: row.to}), (sc:SmartContract {contractAddress: row.contractAddress}), (b:Block {blockNumber: toInteger(row.blockNumber)})
    CREATE (from)-[:TRANSFER {hash: row.hash, value: toFloat(row.value), gas: toInteger(row.gas), gasPrice: toInteger(row.gasPrice), isError: toBoolean(row.isError)}]->(to),
           (from)-[:INVOKES]->(sc),
           (to)-[:INVOKES]->(sc),
           (b)-[:CONTAINS]->(from),
           (b)-[:CONTAINS]->(to)
    """

    driver = GraphDatabase.driver(URI, auth=(USERNAME, PASS))

    with driver.session() as session:
        try:
            session.run(cypher_query, csv_file=csv_file)
            print(f"Data imported from CSV file '{csv_file}' successfully.")
            logging.info(f"Data imported from CSV file '{csv_file}' successfully.")

        except Exception as e:
            print(f"Error importing data from CSV file '{csv_file}': {e}")
            logging.error(f"Error importing data from CSV file '{csv_file}': {e}")

        finally:
            driver.close()
            logging.info("Neo4j driver connection closed.")

## Pushing to Neo4j - Execution

### Illicit Smart Contracts

In [192]:
csv_file = "file:///illicit_transactions_eth.csv"
load_data_from_csv(csv_file)

Data imported from CSV file 'file:///illicit_transactions_eth.csv' successfully.


In [193]:
def check_data_existence():
    # Define Cypher queries to count nodes and relationships
    node_query = "MATCH (n:Account) RETURN COUNT(n:Account) AS nodeCount"
    relationship_query = "MATCH ()-[r]->() RETURN COUNT(r) AS relationshipCount"
    
    # Connect to Neo4j
    driver = GraphDatabase.driver(URI, auth=(USERNAME, PASS))
    
    with driver.session() as session:
        # Execute node count query
        node_result = session.run(node_query).single()
        node_count = node_result["nodeCount"]
        
        # Execute relationship count query
        relationship_result = session.run(relationship_query).single()
        relationship_count = relationship_result["relationshipCount"]
        
        # Print node and relationship counts
        print(f"Node count: {node_count}")
        print(f"Relationship count: {relationship_count}")
        
        # Check if any nodes or relationships exist
        if node_count > 0 or relationship_count > 0:
            print("Data exists in Neo4j.")
        else:
            print("No data found in Neo4j.")

    # Close the Neo4j driver
    driver.close()


In [190]:
check_data_existence()

Node count: 0
Relationship count: 0
No data found in Neo4j.
